In [132]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import time
import re
import numpy as np
import nltk
import pickle
import re
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from afinn import Afinn
from gensim.models.word2vec import Word2Vec

In [46]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [47]:
pd.set_option('mode.chained_assignment',  None)

In [48]:
nbs = pd.read_excel('nbs_data.xlsx')
nbs.head()

,Authors,Author(s) ID,Title,Year,Source title,Affiliations,Authors with affiliations,Abstract,Author Keywords,Index Keywords,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,"Jouin C., Mlocek Z.",57219668257;57219668269;,Evolutions of nature based solutions for storm...,2020,Techniques - Sciences - Methodes,"Atelier Ld, Paris, France","Jouin, C., Atelier Ld, Paris, France; Mlocek, ...",Stormwater management is one of the greatest c...,NaN,cost-benefit analysis; maintenance; project as...,NaN,NaN,NaN,NaN,NaN
1,Buzási A.,56692966000;,Climate vulnerability and adaptation challenge...,2021,Climate,"Department of Environmental Economics, Budapes...","Buzási, A., Department of Environmental Econom...",Wine producers face several challenges regardi...,Adaptation; Lock-in; Viticulture; Vulnerability,NaN,NaN,NaN,NaN,NaN,NaN
2,"Yu G.-R., Yang M., Chen Z., Zhang L.-M.",55807480400;55703268000;57001631400;55077923500;,Technical approach and strategic plan for larg...,2021,Chinese Journal of Applied Ecology,Synthesis Research Center of Chinese Ecosystem...,"Yu, G.-R., Synthesis Research Center of Chines...",Terrestrial ecosystems are the core components...,Ecocivilization; Ecological security pattern; ...,ecological approach; environmental issue; envi...,NaN,NaN,NaN,NaN,NaN
3,"Thorn J.P.R., Biancardi Aleu R., Wijesinghe A....",56613034600;57260808600;57226747891;5726048210...,Mainstreaming nature-based solutions for clima...,2021,Landscape and Urban Planning,"African Climate and Development Initiative, Un...","Thorn, J.P.R., African Climate and Development...",(Despite a growing recognition of the importan...,NaN,climate change; infrastructure planning; natur...,NaN,NaN,NaN,NaN,NaN
4,"Schuerch M., Spencer T., Temmerman S., Kirwan ...",54783506900;55156903100;6602902860;10044268300...,Future response of global coastal wetlands to ...,2018,Nature,"Lincoln Centre for Water and Planetary Health,...","Schuerch, M., Lincoln Centre for Water and Pla...",The response of coastal wetlands to sea-level ...,NaN,coastal waters; letter; occupation; sea level ...,NaN,NaN,NaN,NaN,NaN


In [49]:
df = nbs[['Year','Abstract']] 
df.head()

,Year,Abstract
0,2020,Stormwater management is one of the greatest c...
1,2021,Wine producers face several challenges regardi...
2,2021,Terrestrial ecosystems are the core components...
3,2021,(Despite a growing recognition of the importan...
4,2018,The response of coastal wetlands to sea-level ...


In [9]:
# 2016년 이후 논문만 추출 
df = df[df['Year'] >= 2016]

In [12]:
df = df.reset_index(drop = True)
len(df)

1037

In [13]:
# 영문자 이외 문자(한글, 숫자, 특수문자)는 공백으로 변환
df['Abstract'] = df['Abstract'].str.replace('[^a-zA-Z]',' ',regex=True)

# 고유명사, 첫시작 대문자를 고려하여 모두 소문자로 변경 
df['Abstract'] = df['Abstract'].str.lower()
df.head()

,Year,Abstract
0,2020,stormwater management is one of the greatest c...
1,2021,wine producers face several challenges regardi...
2,2021,terrestrial ecosystems are the core components...
3,2021,despite a growing recognition of the importan...
4,2018,the response of coastal wetlands to sea level ...


In [16]:
df['Abstract_token'] = None
df['Abstract_NN'] = None

for i in range(len(df)):
    df['Abstract_token'][i] = nltk.word_tokenize(df['Abstract'][i])   # Word Tokenization
    df['Abstract_token'][i] = nltk.pos_tag(df['Abstract_token'][i])   # POS tagging
    NN_words = []

    for word, pos in df['Abstract_token'][i]:
        if 'NN' in pos:
            NN_words.append(word)
        df['Abstract_NN'][i] = NN_words   # Extract noun

df.head()

,Year,Abstract,Abstract_token,Abstract_NN
0,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, management, challenges, environme..."
1,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producers, challenges, climate, change,..."
2,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystems, core, components, life, productio..."
3,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos..."
4,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetlands, rise, twenty, century, sc..."


In [17]:
# Lemmaization(원형찾기) => 원형을 찾아 같은 의미의 단어 토큰들을 하나의 값으로 인지하도록 한다. 
wlem = nltk.WordNetLemmatizer()

for i in range(len(df)):

    lemmatized_words = []

    for word in df['Abstract_NN'][i]:
        new_word = wlem.lemmatize(word)
        lemmatized_words.append(new_word)
        df['Abstract_NN'][i] = lemmatized_words
        
df.head()

,Year,Abstract,Abstract_token,Abstract_NN
0,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, management, challenge, environmen..."
1,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, change, i..."
2,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,..."
3,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos..."
4,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca..."


In [18]:
# 1글자 제거 
df['Abstract_NN'] = df['Abstract_NN'].apply(lambda x: [word for word in x if (len(word) > 1 )])

df.head()

,Year,Abstract,Abstract_token,Abstract_NN
0,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, management, challenge, environmen..."
1,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, change, i..."
2,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,..."
3,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos..."
4,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca..."


In [19]:
#nltk에서 제공하는 불용어사전 이용
stopwords_list = stopwords.words('english')

In [20]:
# 사용자 정의 불용어 사전 
customized_stopwords = ['nature', 'based', 'solutions', 'nbs', 'studing', 'studies', 'study','management', 'manage', 'model', 'use', 
                        'plan', 'increase','service',  'result', 'system', 'implement','implementation', 'treat', 'treatment']

In [21]:
stopwords_list.extend(customized_stopwords)

In [22]:
for i in range(len(df)):
    unique_NN_words = set(df['Abstract_NN'][i])
    final_NN_words = df['Abstract_NN'][i]

    for word in unique_NN_words:
        if word in stopwords_list:
            while word in final_NN_words: final_NN_words.remove(word)   # 불용어 제거 
    df['Abstract_NN'][i] = final_NN_words

In [23]:
df.head()

,Year,Abstract,Abstract_token,Abstract_NN
0,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, challenge, environment, way, envi..."
1,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, change, i..."
2,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,..."
3,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos..."
4,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca..."


In [24]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [25]:
df['Abstract_str'] = None

for i in range(len(df)):
    df['Abstract_str'][i] = listToString(df['Abstract_NN'][i])

In [35]:
df.head()

,Year,Abstract,Abstract_token,Abstract_NN,Abstract_str,tfidf_NN,제거개수
0,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, challenge, environment, way, envi...",stormwater challenge environment way environme...,"[stormwater, challenge, environment, way, envi...",10
1,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, change, i...",wine producer challenge climate change industr...,"[challenge, climate, change, industry, vulnera...",13
2,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,...",ecosystem core component life production activ...,"[ecosystem, core, component, life, production,...",7
3,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos...",recognition importance infrastructure ecosyste...,"[recognition, importance, infrastructure, ecos...",12
4,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca...",response wetland rise twenty century scale pro...,"[response, wetland, rise, century, scale, proj...",15


In [37]:
vectorizer = TfidfVectorizer(stop_words="english",
                            use_idf=True,
                            ngram_range=(1,1), # 오직 1-grams만 고려함
                            min_df = 0.01,     # 1% 미만으로 문서에서 출현하는 단어들은 삭제
                            max_df = 0.95)     # 전체 문서의 95% 문서에 출현하는 단어들은 너무 빈번한 단어라고 판단하여 삭제  

In [38]:
tfidf = vectorizer.fit_transform(df['Abstract_str'])

In [39]:
tf_idf_list = vectorizer.get_feature_names()
tf_idf_list

['ability',
 'absence',
 'abundance',
 'academy',
 'acceptance',
 'access',
 'accessibility',
 'account',
 'accounting',
 'accumulation',
 'achievement',
 'action',
 'activity',
 'actor',
 'adaptation',
 'addition',
 'adoption',
 'advantage',
 'afforestation',
 'africa',
 'age',
 'agency',
 'agenda',
 'agent',
 'agreement',
 'agriculture',
 'aim',
 'air',
 'alternative',
 'america',
 'analysis',
 'applicability',
 'application',
 'approach',
 'area',
 'arrangement',
 'art',
 'article',
 'asia',
 'aspect',
 'assessment',
 'asset',
 'association',
 'assumption',
 'attention',
 'attenuation',
 'attitude',
 'attribute',
 'australia',
 'author',
 'authority',
 'availability',
 'average',
 'awareness',
 'background',
 'balance',
 'bank',
 'barrier',
 'base',
 'basel',
 'baseline',
 'basin',
 'basis',
 'beach',
 'behavior',
 'beneficiary',
 'benefit',
 'biodiversity',
 'biomass',
 'blue',
 'bod',
 'body',
 'boundary',
 'brazil',
 'breakwater',
 'budget',
 'buffer',
 'building',
 'business',
 

In [40]:
df['tfidf_NN'] = None 

for i in range(len(df)):
    result = pd.DataFrame(df['Abstract_NN'][i])
    result = result[result[0].str.contains('|'.join(tf_idf_list))][0].tolist()   # tf_idf_list에 해당되는 단어만 남겨두기 
    df['tfidf_NN'][i] = result

In [41]:
df['제거개수'] = None

for i in range(len(df)):
    df['제거개수'][i] = len(df['Abstract_NN'][i]) - len(df['tfidf_NN'][i])

In [42]:
df.head()

,Year,Abstract,Abstract_token,Abstract_NN,Abstract_str,tfidf_NN,제거개수
0,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, challenge, environment, way, envi...",stormwater challenge environment way environme...,"[stormwater, challenge, environment, way, envi...",10
1,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, change, i...",wine producer challenge climate change industr...,"[challenge, climate, change, industry, vulnera...",13
2,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,...",ecosystem core component life production activ...,"[ecosystem, core, component, life, production,...",7
3,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos...",recognition importance infrastructure ecosyste...,"[recognition, importance, infrastructure, ecos...",12
4,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca...",response wetland rise twenty century scale pro...,"[response, wetland, rise, century, scale, proj...",15


In [43]:
noun_list = df['tfidf_NN'].tolist()

In [44]:
print(noun_list)

[['stormwater', 'challenge', 'environment', 'way', 'environment', 'constraint', 'multitude', 'issue', 'space', 'connection', 'people', 'living', 'environment', 'land', 'value', 'development', 'maintenance', 'cost', 'biodiversity', 'heath', 'island', 'effect', 'designer', 'tool', 'order', 'water', 'city', 'project', 'aim', 'article', 'number', 'case', 'project', 'villeneuve', 'solution', 'solution', 'city', 'project', 'case', 'starting', 'point', 'discussion', 'designer', 'publication', 'feedback', 'legislation', 'water', 'preconception', 'regard', 'efficiency', 'cost', 'maintenance', 'order', 'stormwater', 'designer', 'role', 'water', 'stormwater', 'rain', 'site', 'multifunctionality', 'space'], ['challenge', 'climate', 'change', 'industry', 'vulnerability', 'assessment', 'climate', 'research', 'paper', 'aim', 'component', 'climate', 'vulnerability', 'region', 'adaptation', 'technique', 'term', 'sustainability', 'aspect', 'analysis', 'area', 'challenge', 'drought', 'period', 'sensitivi

In [67]:
keyword = ["forest", "forestation", "afforestation", "reforestation", "green", "tree"]

In [75]:
# keyword가 들어있는 논문만 추출
noun_list[0][1]
final = []
flag = 0
for i in range(len(noun_list)):
    for j in range(len(noun_list[i])):
        for k in range(len(keyword)):
            if keyword[k] in noun_list[i][j]:
                flag = 1
    if (flag == 1):
        final.append(noun_list[i])
        flag = 0

### 장바구니 분석

In [76]:
# 데이터 셋 생성 및 가공
te = TransactionEncoder()
te_ary = te.fit(final).transform(final)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [77]:
df.head()

,ability,aboveground,absence,abstraction,abundance,acceptability,acceptance,access,accessibility,accommodate,...,workshop,world,worldwide,wtp,year,yield,yielding,yr,yrd,zone
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [82]:
# Apriori 알고리즘 활용
frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)

In [83]:
frequent_itemsets

,support,itemsets
0,0.469565,(area)
1,0.469565,(author)
2,0.321739,(benefit)
3,0.469565,(change)
4,0.395652,(city)
5,0.413043,(climate)
6,0.430435,(ecosystem)
7,0.373913,(forest)
8,0.313043,(impact)
9,0.760870,(solution)


In [85]:
association_rules(frequent_itemsets, metric="lift", min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(area),(solution),0.469565,0.760870,0.360870,0.768519,1.010053,0.003592,1.033043
1,(solution),(area),0.760870,0.469565,0.360870,0.474286,1.010053,0.003592,1.008979
2,(change),(climate),0.469565,0.413043,0.321739,0.685185,1.658869,0.127788,1.864450
3,(climate),(change),0.413043,0.469565,0.321739,0.778947,1.658869,0.127788,2.399586
4,(city),(solution),0.395652,0.760870,0.304348,0.769231,1.010989,0.003308,1.036232
5,(solution),(city),0.760870,0.395652,0.304348,0.400000,1.010989,0.003308,1.007246
6,(climate),(solution),0.413043,0.760870,0.317391,0.768421,1.009925,0.003119,1.032609
7,(solution),(climate),0.760870,0.413043,0.317391,0.417143,1.009925,0.003119,1.007033


### 연관어 분석

In [114]:
abstract_df = nbs[['Abstract']] 
abstract = abstract_df.iloc[:,0]

In [115]:
# 동시 출현 기반 연관어 분석
tokenizer = RegexpTokenizer('[\w]+')
stop_words = stopwords.words('english')

count = {}
for line in abstract:
    words = line.lower()
    tokens = tokenizer.tokenize(words)
    stopped_tokens = [i for i in list(set(tokens)) if not i in stop_words+["br"]]
    stopped_tokens_2 = [i for i in stopped_tokens if len(i) > 1]
    for i, a in enumerate(stopped_tokens_2):
        for b in stopped_tokens_2[i+1:]:
            if a > b:
                count[b, a] = count.get((b, a), 0) + 1
            else:
                count[a, b] = count.get((a, b), 0) + 1

In [116]:
df = pd.DataFrame.from_dict(count, orient='index')
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
Index: 3637148 entries, ('issues', 'one') to ('closure', 'content')
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   0       int64
dtypes: int64(1)
memory usage: 55.5+ MB
None


,0
"(issues, one)",14
"(one, perfectly)",1
"(obstacles, one)",3
"(city, one)",35
"(aim, one)",17
...,...
"(model, rich)",1
"(closure, rich)",1
"(canopy, rich)",1
"(canopy, closure)",1


In [117]:
list1 = []
for i in range(len(df)):
    list1.append([df.index[i][0], df.index[i][1], df[0][i]])
df2 = pd.DataFrame(list1, columns = ['term1', 'term2', 'freq'])
df3 = df2.sort_values(by=['freq'], ascending = False)
df3 = df3.reset_index(drop=True)
df3

,term1,term2,freq
0,based,nature,836
1,based,solutions,688
2,nature,solutions,683
3,based,study,402
4,based,urban,398
...,...,...,...
3637143,delineate,part,1
3637144,delineate,space,1
3637145,delineate,sandy,1
3637146,allowing,delineate,1


In [122]:
# 통계적 가중치 기반 연관어 분석
stop_words = stopwords.words('english')
vec = TfidfVectorizer(stop_words=stop_words)
vector_abstract = vec.fit_transform(abstract)
vector_abstract

<1056x13126 sparse matrix of type '<class 'numpy.float64'>'
	with 120284 stored elements in Compressed Sparse Row format>

In [123]:
A = vector_abstract.toarray()
pd.DataFrame(A)

,0,1,2,3,4,5,6,7,8,9,...,13116,13117,13118,13119,13120,13121,13122,13123,13124,13125
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.141664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
A=A.transpose()
pd.DataFrame(A)

,0,1,2,3,4,5,6,7,8,9,...,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [125]:
A_sparse = sparse.csr_matrix(A) # A를 다시 희소행렬로 변환
similarities_sparse = cosine_similarity(A_sparse, dense_output=False)
list(similarities_sparse.todok().items())[35000:35010]

[((8115, 86), 0.015176393787488123),
 ((8118, 86), 0.06676600556698356),
 ((8125, 86), 0.018770776209032444),
 ((8136, 86), 0.16480650595478258),
 ((8142, 86), 0.05783564242286643),
 ((8144, 86), 0.03236665927050899),
 ((8145, 86), 0.1367452837133005),
 ((8166, 86), 0.030207532217062846),
 ((8180, 86), 0.01239970454574805),
 ((8181, 86), 0.056590536842274476)]

In [128]:
df = pd.DataFrame(list(similarities_sparse.todok().items()), columns=['words', 'weight'])
df2 = df.sort_values(by=['weight'], ascending=False)
df2 = df2.reset_index(drop=True)
df3 = df2.loc[np.round(df2['weight']) < 1] # 자신 끼리의 페어는 유사도 1
df3 = df3.reset_index(drop=True)

df3.head(10)

,words,weight
0,"(339, 7766)",0.499997
1,"(4295, 7766)",0.499997
2,"(7766, 4295)",0.499997
3,"(2382, 7766)",0.499997
4,"(7766, 2382)",0.499997
5,"(7766, 339)",0.499997
6,"(391, 5571)",0.499992
7,"(5571, 391)",0.499992
8,"(6855, 6373)",0.499983
9,"(11055, 6373)",0.499983


In [131]:
print(vec.get_feature_names()[4295])
print(vec.get_feature_names()[7766])

enthusiasm
motivations


In [133]:
# word2vec 기반 연관어 분석
tokenizer = RegexpTokenizer('[\w]+')
stop_words = stopwords.words('english')

text = [] # word2vec을 적용할 list
for line in abstract:
    words = line.lower() # 각 리뷰를 소문자로 변환
    tokens = tokenizer.tokenize(words) # 각 리뷰를 토큰화한 뒤 리스트에 저장
    stopped_tokens = [i for i in list(set(tokens)) if not i in stop_words+["br"]]
    stopped_tokens2 = [i for i in stopped_tokens if len(i)>1]
    text.append(stopped_tokens2)

In [135]:
model = Word2Vec(text, sg=1, window=2, min_count=3)
model.init_sims(replace=True)
model.wv.most_similar("forest", topn=5)

[('trees', 0.9988880157470703),
 ('major', 0.9983332753181458),
 ('sediment', 0.9982197284698486),
 ('strategies', 0.9976354837417603),
 ('evaluate', 0.996991753578186)]